In [2]:
!pip install langchain langchain-experimental openai pymysql

# Importing Necessary Packages

In [3]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [4]:
OPENAI_API_KEY = "Your API Key"
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

C:\Users\akhil\AppData\Local\Temp\ipykernel_12536\1703113971.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


# Database Setup

In [15]:
host = 'localhost'
port = '3306'
username = 'root'
# password = rekubannu
database_schema = 'mydb'
mysql_uri = f"mysql+pymysql://{username}@{host}:{port}/{database_schema}"
# mysql_uri = f"mysql+pymysql://{username:rekubannu}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri, include_tables=['training_data'],sample_rows_in_table_info=2)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [16]:
def retrieve_from_db(query: str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

In [17]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    
    system_message = """you are a finance, data analysis, and risk management.
        your task is to answer users' questions by providing relevant information from a database of training data.
        Tha database contaims the following information for each id:
        -Id
        -Income
        -Age
        -Experience
        -Married
        -House_Ownership
        -Car_Ownership
        -Profession
        -CITY
        -STATE
        -CURRENT_JOB_YRS
        -CYRRENT_HOUSE_YRS
        -Risk_Flag

        Example:

        Input:
        who has the most experience in their field?

        context:
        The most experience individual is following details:
        Income: ₹1,797,876
        Age: 76
        Experience: 20 years
        Marital Status: Single
        House Ownership: No rent, no own
        Car Ownership: No
        Profession: Mechanical Engineer
        City: Erode
        State: Tamil Nadu
        Current Job Years: 11
        Current House Years: 14
        Risk Flag: 0 (Low Risk)

        Output:
        The most experienced individual is Mechanical Engineer from TamilNadu
        """
    
    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        
        Context:
        {db_context}
        
        Output:
        """
    )
    messages = [
      SystemMessage(content=system_message),
      human_qry_template.format(human_input=query, db_context=db_context)
    ]
    response = llm(messages).content
    return response

# Ask the queries

In [18]:
generate("who has the most experince in the given data?")

C:\Users\akhil\AppData\Local\Temp\ipykernel_12536\3606437300.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  db_context = db_chain(query)




> Entering new SQLDatabaseChain chain...
who has the most experince in the given data?
SQLQuery:SELECT `Profession`, `Experience`
FROM training_data
ORDER BY `Experience` DESC
LIMIT 1;
SQLResult: [('Mechanical_engineer', 20)]
Answer:The person with the most experience in the given data is a Mechanical Engineer with 20 years of experience.
> Finished chain.


C:\Users\akhil\AppData\Local\Temp\ipykernel_12536\3082036862.py:59: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages).content


'The most experienced individual in the given data is a Mechanical Engineer with 20 years of experience.'

In [ ]:
generate("who has the least expereince ?")



> Entering new SQLDatabaseChain chain...
who has the  least expereince ?
SQLQuery:SELECT `Profession`, `Experience` 
FROM training_data 
ORDER BY `Experience` ASC 
LIMIT 1;
SQLResult: [('Civil_servant', 0)]
Answer:The person with the least experience is a Civil Servant with 0 years of experience.
> Finished chain.


'The person with the least experience is a Civil Servant.'

In [26]:
generate("who is having the highest income?")



> Entering new SQLDatabaseChain chain...
who is having the highest income?
SQLQuery:SELECT `Profession`, `Income` 
FROM training_data 
ORDER BY `Income` DESC 
LIMIT 1;
SQLResult: [('Drafter', 9999180)]
Answer:The person with the highest income is a Drafter with an income of 9999180.
> Finished chain.


'The person with the highest income is a Drafter.'

In [33]:
generate("how many poeple have the rented house in the given dataset ?")



> Entering new SQLDatabaseChain chain...
how many poeple have the rented house in the given dataset ?
SQLQuery:SELECT COUNT(*) AS Num_Rented_House
FROM training_data
WHERE House_Ownership = 'rented';
SQLResult: [(2299,)]
Answer:There are 2299 people who have a rented house in the given dataset.
> Finished chain.


'There are 2299 people who have a rented house in the given dataset.'

In [34]:
generate("how many poeple have the own house in the given dataset ?")



> Entering new SQLDatabaseChain chain...
how many poeple have the own house in the given dataset ?
SQLQuery:SELECT COUNT(*) AS Num_People_Own_House
FROM training_data
WHERE House_Ownership = 'owned';
SQLResult: [(129,)]
Answer:There are 129 people who own a house in the given dataset.
> Finished chain.


'There are 129 people who own a house in the given dataset.'

In [36]:
generate("how many poeple have the 10 years experinece in the given dataset and provide me eaach one details  ?")



> Entering new SQLDatabaseChain chain...
how many poeple have the 10 years experinece in the given dataset and provide me eaach one details  ?
SQLQuery:SELECT `Id`, `Income`, `Age`, `Experience`, `Married`, `House_Ownership`, `Car_Ownership`, `Profession`, `CITY`, `STATE`, `CURRENT_JOB_YRS`, `CURRENT_HOUSE_YRS`, `Risk_Flag`
FROM training_data
WHERE `Experience` = 10
LIMIT 5;
SQLResult: [(2, 7574516, 40, 10, 'single', 'rented', 'no', 'Software_Developer', 'Parbhani', 'Maharashtra', 9, 13, 0), (33, 4128828, 21, 10, 'single', 'rented', 'no', 'Computer_hardware_engineer', 'Khammam', 'Telangana', 10, 12, 0), (34, 5023035, 26, 10, 'single', 'rented', 'yes', 'Petroleum_Engineer', 'Madurai', 'Tamil_Nadu', 9, 13, 0), (48, 8477026, 52, 10, 'single', 'rented', 'no', 'Design_Engineer', 'Bijapur', 'Karnataka', 10, 10, 0), (73, 2408148, 59, 10, 'single', 'rented', 'no', 'Architect', 'Solapur', 'Maharashtra', 7, 11, 0)]
Answer:There are 5 people with 10 years of experience in the dataset. Their det

'There are 5 people with 10 years of experience in the dataset. Their details are as follows:\n\n1. Id: 2\n   Income: ₹7,574,516\n   Age: 40\n   Married: Single\n   House Ownership: Rented\n   Car Ownership: No\n   Profession: Software Developer\n   City: Parbhani\n   State: Maharashtra\n   Current Job Years: 9\n   Current House Years: 13\n   Risk Flag: 0 (Low Risk)\n\n2. Id: 33\n   Income: ₹4,128,828\n   Age: 21\n   Married: Single\n   House Ownership: Rented\n   Car Ownership: No\n   Profession: Computer Hardware Engineer\n   City: Khammam\n   State: Telangana\n   Current Job Years: 10\n   Current House Years: 12\n   Risk Flag: 0 (Low Risk)\n\n3. Id: 34\n   Income: ₹5,023,035\n   Age: 26\n   Married: Single\n   House Ownership: Rented\n   Car Ownership: Yes\n   Profession: Petroleum Engineer\n   City: Madurai\n   State: Tamil Nadu\n   Current Job Years: 9\n   Current House Years: 13\n   Risk Flag: 0 (Low Risk)\n\n4. Id: 48\n   Income: ₹8,477,026\n   Age: 52\n   Married: Single\n   Ho

In [37]:
generate("how many poeple have the risk flag in the given dataset ?")



> Entering new SQLDatabaseChain chain...
how many poeple have the risk flag in the given dataset ?
SQLQuery:SELECT COUNT(`Id`) AS Total_People_With_Risk_Flag
FROM training_data
WHERE `Risk_Flag` = 1;
SQLResult: [(441,)]
Answer:There are 441 people with the risk flag in the given dataset.
> Finished chain.


'There are 441 people with the risk flag in the given dataset.'